In [6]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

TRIPS_ROWS_LIMIT = None
TYPE_TRIPS = 'trips'
TYPE_STATIONS = 'stations'
TYPE_WEATHER = 'weather'
        
cities = set()
paths_by_city_and_type = defaultdict(dict)
for dirname, _, filenames in os.walk('../client/data/'):
    for filename in filenames:
        city = os.path.basename(dirname)
        cities.add(city)
        file_type = os.path.splitext(filename)[0]
        paths_by_city_and_type[city][file_type] = os.path.join(dirname, filename)
            
print(cities)

{'montreal', 'toronto', 'washington'}


In [ ]:
import datetime 
weather_by_city = {}
for city in cities:
    weather = pd.read_csv(paths_by_city_and_type[city][TYPE_WEATHER], delimiter=',')
    weather = weather[['date', 'prectot']]
    weather['date']= pd.to_datetime(weather['date']).apply(lambda t: (t - datetime.timedelta(days=1)).date())
    weather_by_city[city] = weather
    
stations_by_city = {}
for city in cities:
    stations = pd.read_csv(paths_by_city_and_type[city][TYPE_STATIONS], delimiter=',')
    stations_by_city[city] = stations

trips_list = []
for city in cities:
    city_trips = pd.read_csv(paths_by_city_and_type[city][TYPE_TRIPS], delimiter=',', nrows=TRIPS_ROWS_LIMIT)
    city_trips.loc[city_trips['duration_sec'] < 0 , 'duration_sec'] = 0 
    city_trips['start_date']= pd.to_datetime(city_trips['start_date'])
    city_trips['end_date']= pd.to_datetime(city_trips['end_date'])
    city_trips['start_date_day'] = city_trips['start_date'].apply(lambda t:t.date())
    city_trips = city_trips.merge(weather_by_city[city], left_on='start_date_day', right_on='date')
    city_trips = city_trips.merge(stations_by_city[city], left_on=['start_station_code', 'yearid'], right_on=['code', 'yearid'])
    city_trips.rename(columns = {'name':'start_station_name', 'latitude': 'start_station_latitude', 'longitude': 'start_station_longitude', }, inplace = True)
    city_trips = city_trips.merge(stations_by_city[city], left_on=['end_station_code', 'yearid'], right_on=['code', 'yearid'])
    city_trips.rename(columns = {'name':'end_station_name', 'latitude': 'end_station_latitude', 'longitude': 'end_station_longitude', }, inplace = True)
    city_trips['city'] = city
    city_trips = city_trips.drop(columns=['start_date', 'start_station_code', 'end_date', 'end_station_code', 'is_member', 'code_x', 'code_y', 'date'])
    trips_list.append(city_trips)
    
trips = pd.concat(trips_list)
trips_list = None
print(trips.size)   
print(trips.tail())   

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


730662933
          duration_sec  yearid start_date_day  prectot  \
27770337        2010.0    2020     2020-06-09     4.85   
27770338         329.0    2020     2020-03-06     0.10   
27770339         310.0    2020     2020-01-31     1.46   
27770340         722.0    2020     2020-06-02     0.03   
27770341         556.0    2020     2020-01-29     0.00   

                         start_station_name  start_station_latitude  \
27770337    Key West Ave & Great Seneca Hwy                     NaN   
27770338  Traville Gateway Dr & Gudelsky Dr                     NaN   
27770339               Shady Grove Hospital                     NaN   
27770340               Shady Grove Hospital                     NaN   
27770341   Medical Center Dr & Key West Ave                     NaN   

          start_station_longitude          end_station_name  \
27770337                      NaN  Broschart & Blackwell Rd   
27770338                      NaN  Broschart & Blackwell Rd   
27770339                 

In [3]:
#Query 1 - Average duration on trips during >20mm precipitation days
trips[trips.prectot > 30].groupby(['start_date_day'])['duration_sec'].mean()

start_date_day
2011-03-05    1667.362707
2011-03-09     830.043362
2011-04-15    1281.006812
2011-08-26    1059.071630
2011-09-04    2159.195062
                 ...     
2020-08-03    2197.959854
2020-10-11    1439.637820
2020-10-28    2156.449094
2020-11-10    1406.033608
2020-11-29    1987.024564
Name: duration_sec, Length: 63, dtype: float64

In [4]:
#Query 2 - 
station_trips_totals = trips.groupby(['start_station_name', 'yearid']).size().reset_index(name='qty_trips')
station_trips_2016 = station_trips_totals[station_trips_totals.yearid == 2016][['start_station_name', 'qty_trips']]
station_trips_2017 = station_trips_totals[station_trips_totals.yearid == 2017][['start_station_name', 'qty_trips']]
station_trips = station_trips_2017.merge(station_trips_2016, left_on='start_station_name', right_on='start_station_name', suffixes=('_qty_2017','_qty_2016'))
station_trips[station_trips.qty_trips_qty_2017 > 2*station_trips.qty_trips_qty_2016]

,start_station_name,qty_trips_qty_2017,qty_trips_qty_2016
22,12th St & Pennsylvania Ave SE,4925,47
33,14th & Irving St NW,22389,8321
57,16th & Q St SE / Anacostia HS,711,151
63,17th St & Independence Ave SW,30088,13733
81,1st & D St SE,13741,3204
...,...,...,...
935,Yarmouth Rd / Christie St,2917,907
936,Yonge St / Aylmer Ave,1923,728
938,Yonge St / Wood St,7564,3320
939,Yonge St / Yorkville Ave,6462,2391


In [5]:
#Query 3 - Stations with more than 6km avg to arrive at them
from haversine import haversine

montreal_trips = trips[trips.city == 'montreal']
trips = None
station_distances_array = montreal_trips.apply(lambda r : [r['end_station_name'], haversine((r['start_station_latitude'], r['start_station_longitude']), (r['end_station_latitude'], r['end_station_longitude']))], axis = 1)
montreal_trips = None
station_distances = pd.DataFrame(station_distances_array.values.tolist(), columns=['end_station_name', 'distance'])

montreal_station_mean_distances = station_distances.groupby(['end_station_name'])['distance'].mean().reset_index()
montreal_station_mean_distances[montreal_station_mean_distances.distance >= 6]

,end_station_name,distance
943,de Grosbois / Ducheneau,10.562033
